In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import ase
import glob
import tqdm
import time
import scipy
import numba
import pickle
import numpy as np
import pandas as pd
import ase.io as aio
import seaborn as sns
import scipy.io as sio
import multiprocessing as mp
from matplotlib import colors
import matplotlib.pyplot as plt
import poremks.porosity as pore
from scipy.spatial import cKDTree
from collections import defaultdict
import poremks.correlations as corr
import poremks.grid_generator as gen
from poremks.helpers import write2vtk
from toolz.curried import pipe, curry
from scipy.sparse.csgraph import dijkstra

In [3]:
def saver(fname, obj):
    with open(fname, "wb") as f:
        pickle.dump(obj, f)
        
def loader(fname):
    with open(fname, "rb") as f:
        return pickle.load(f)

In [4]:
conv = curry(np.convolve)(mode="valid")

def dist(f1, f2):
    return conv(f1,f2).max()/np.sqrt(conv(f1,f1)[0] * conv(f2,f2)[0])

def dmatrix(items):
    mat = np.zeros((len(items), len(items)))
    for i1, item1 in enumerate(items):
        for i2, item2 in enumerate(items):
            mat[i1, i2] = dist(item1, item2)
    return mat

def get_radii(atom_id, radius_type="vdw"):
    """
    atom_id: element symbol
    radius_type = "vdw" for Van der Waals or "cov" for Covalent
    """
    import pandas as pd
    
    xl = pd.ExcelFile("Elemental_Radii.xlsx")
    df = xl.parse(sheet_name=0, header = 2, index_col=1)
    
    if radius_type is "cov":
        key = 6
    elif radius_type is "vdw":
        key = 7
    else:
        raise ValueError("radius_type not supported")
    if atom_id in df.index:
        return df.loc[atom_id][key]
    else:
        raise ValueError("Elemental symbol not found")
        
def get_pathLength(path, coords):
    l = 0.0
    coord0 = coords[path[0]]
    for i, idx in enumerate(path[1:]):
        coord = coords[idx]
        l = l + np.sqrt(np.sum((coord - coord0)**2))
        coord0 = coord
    return l

@numba.njit
def pred_search(pred, path, idx0):
    done = False
    idx = path[0]
    while not done:
        idx = pred[idx]

        if idx == -9999:
            break
        path.append(idx)
        if idx == idx0:
            done = True
    return path, done

@curry
def gen_throughPath(S, depth):
    coords = np.concatenate([np.where(S>0)], axis=1).T
    i=0
    bot = []
    top = []

    
    l = S.shape[2]

    S_1 = np.zeros(S.shape, dtype=np.uint8)

    for i in range(depth):
        bot = bot + list(np.where(coords[:,2] == i)[0])
        top = top + list(np.where(coords[:,2] == (S.shape[2]-1-i))[0])

    tree = cKDTree(coords)
    dok_mat = tree.sparse_distance_matrix(tree, max_distance=2, p=2.0)
    dist_mat, pred = dijkstra(dok_mat,
                              directed=False,
                              indices=top,
                              return_predecessors=True,
                              unweighted=True)
    tort_list = []
    path_list = []
    
    for index, idx0 in enumerate(top):
        pred0 = pred[index,:]
        for idx in bot:
            path = [idx,]

            path, done = pred_search(pred0, path, idx0)

            if done:
                path_list.append([coords[path][:,idim] for idim in range(3)])
                dx, dy, dz = path_list[-1]
                S_1[dx, dy, dz] += 1
                tort_list.append(get_pathLength(path, coords)/ l)
                

    return S_1, tort_list, path_list


def func(fname):
    atom = pipe(fname, 
                lambda fname: aio.read(fname), 
                lambda x: x.repeat([2, 2, 1]))
    print(fname)
    atom_ids = sorted(np.unique(atom.get_chemical_symbols()))

    radii = {}
    for idx in atom_ids:
        radii[idx] = get_radii(idx, radius_type="cov")

    print("No. of atoms in the structure: {}".format(len(atom)))

    len_pixel = 10
    padval = ((0, 0), (0, 0), (0, 0))
    S, S_list, box_dim = gen.grid_maker(atom, len_pixel=len_pixel, atomic_radii=radii, full=False, fft=False)
    
    S_dgrid = pipe(S, 
                   lambda s: np.pad(s, padval, 'constant', constant_values=0),  
                   lambda s: pore.dgrid(s, len_pixel=len_pixel))

    S_paths, torts, paths = pipe(S_dgrid, 
                                 lambda s: (pore.gen_cleanPore(s, r_probe=0.2, r_min=2.5, len_pixel=len_pixel) > 0) * 1, 
                                 lambda s: np.pad(s, pad_width=((0,0),(0,0),(len_pixel, len_pixel)), mode = "constant", constant_values=1), 
                                 lambda s: pore.gen_medialAxis(s)[:,:,len_pixel:-len_pixel],
                                 lambda s: gen_throughPath(s, depth=1))

    d = {}
    d["torts"]= np.asarray(torts)
    d["dist_list"] = [S_dgrid[paths[ix]] for ix in range(len(paths))]
    saver(fname="likely-min/"+fname.split("/")[-1][:-4]+"_paths.pkl", obj=d)

In [6]:
# cif = "MFI.cif"
# fname = os.path.join("../structures/iza-bulk-structures/", cif)

# flist1 = sorted(glob.glob("../structures/likely-min-energy-structures/*.cif"))
# flist0 = sorted(glob.glob("likely-min/*.pkl"))

# i0 = [f.split("/")[-1][:-4] for f in flist1]
# i1 = [f.split("/")[-1][:-10] for f in flist0]

# len(set(i0))

# flist = ["../structures/likely-min-energy-structures/"+f+".cif" for f in set(i0) - set(i0).intersection(i1)]

# len(flist)

# n_atoms = []
# for ix, fname in enumerate(flist):
#     atom = pipe(fname, 
#                 lambda fname: aio.read(fname), 
#                 lambda x: x.repeat([2, 2, 1]))
#     n_atoms.append(len(atom))
    
# flist_new = np.array(flist)[np.array(n_atoms)<2000]
# print(flist_new)

In [7]:
flist_new = sorted(glob.glob("../structures/likely-min-energy-structures/*.cif"))

In [10]:
with mp.Pool(processes=12) as p:
    p.map(func, flist_new)

../structures/likely-min-energy-structures/zz_ABWff-[1,0,1]-L-0.732814_0-U-0.748843_0-ss-12.8142965856.cif
../structures/likely-min-energy-structures/zz_BIKff-[1,1,0]-L-0.595321_0-U-0.011766_0-ss-24.5466935749.cif
../structures/likely-min-energy-structures/zz_CANff-[1,0,0]-L-0.081993_0-U-0.922017_0-ss-27.4097978015.cif
../structures/likely-min-energy-structures/zz_AFGff-[1,0,0]-L-0.412401_0-U-0.412521_0-ss-11.4626779114.cif
../structures/likely-min-energy-structures/zz_DACff-[1,0,0]-L-0.077151_0-U-0.923712_0-ss-35.5334111581.cif
../structures/likely-min-energy-structures/zz_CGSff-[0,1,1]-L-0.106958_0-U-0.893229_0-ss-22.1923351913.cif
../structures/likely-min-energy-structures/zz_AWWff-[1,1,0]-L-0.660176_0-U-0.660181_0-ss-19.1957314898.cif
../structures/likely-min-energy-structures/zz_EABff-[1,0,1]-L-0.497628_0-U-0.497453_0-ss-19.8193628093.cif
../structures/likely-min-energy-structures/zz_ATOff-[1,1,0]-L-0.143787_0-U-0.847486_0-ss-20.641902459.cif
../structures/likely-min-energy-struct

../structures/likely-min-energy-structures/zz_CHAff-[1,0,1]-L-0.335274_0-U-0.335461_0-ss-18.3566388438.cif
No. of atoms in the structure: 936
../structures/likely-min-energy-structures/zz_BPHff-[0,0,1]-L-0.121354_0-U-0.878696_0-ss-25.695821.cif
No. of atoms in the structure: 696
../structures/likely-min-energy-structures/zz_ATSff-[0,1,0]-L-0.459988_0-U-0.540127_0-ss-47.4294568295.cif
No. of atoms in the structure: 648
../structures/likely-min-energy-structures/zz_CFIff-[0,0,1]-L-0.466205_0-U-0.533835_0-ss-30.0721150203.cif
No. of atoms in the structure: 456
../structures/likely-min-energy-structures/zz_APDff-[0,2,1]-L-0.3515735_0-U-0.8515735_0-ss-7.09695300884.cif
No. of atoms in the structure: 480
../structures/likely-min-energy-structures/zz_BPHff-[0,0,1]-L-0.121354_0-U-0.878696_0-ss-38.5437315.cif
No. of atoms in the structure: 1032
../structures/likely-min-energy-structures/zz_BREff-[0,1,0]-L-0.084139_0-U-0.916794_0-ss-25.36882335.cif
No. of atoms in the structure: 300
../structure

../structures/likely-min-energy-structures/zz_AENff-[1,0,0]-L-0.0865_0-U-0.91349_0-ss-53.1897124753.cif
No. of atoms in the structure: 1776
../structures/likely-min-energy-structures/zz_CONff-[2,0,1]-L-0.0598405_0-U-0.9402705_0-ss-39.0564765667.cif
No. of atoms in the structure: 2784
../structures/likely-min-energy-structures/zz_ASVff-[1,0,0]-L-6e-05_0-U-5.4e-05_0-ss-29.2321083573.cif
No. of atoms in the structure: 768
../structures/likely-min-energy-structures/zz_AFNff-[0,0,1]-L-0.347305_0-U-0.347305_0-ss-19.38836457.cif
No. of atoms in the structure: 816
../structures/likely-min-energy-structures/zz_ASVff-[1,0,0]-L-6e-05_0-U-5.4e-05_0-ss-37.7254120773.cif
No. of atoms in the structure: 1008
../structures/likely-min-energy-structures/zz_AWOff-[0,0,1]-L-0.080026_0-U-0.920078_0-ss-19.2101691895.cif
No. of atoms in the structure: 624
../structures/likely-min-energy-structures/zz_ATNff-[1,0,0]-L-0.11919_0-U-0.880774_0-ss-12.9613197204.cif
No. of atoms in the structure: 216
../structures/l

No. of atoms in the structure: 264
../structures/likely-min-energy-structures/zz_IMFff-[0,1,0]-L-0.189064_0-U-0.810929_0-ss-151.074674342.cif
No. of atoms in the structure: 9312
../structures/likely-min-energy-structures/zz_CSVff-[0,1,0]-L-0.13114_0-U-0.86904_0-ss-21.7808521389.cif
No. of atoms in the structure: 504
../structures/likely-min-energy-structures/zz_AFYff-[0,0,1]-L-0.182715_0-U-0.817391_0-ss-8.53305392734.cif
No. of atoms in the structure: 216
../structures/likely-min-energy-structures/zz_AHTff-[1,1,0]-L-0.164397_0-U-0.828914_0-ss-16.6775453701.cif
No. of atoms in the structure: 624
../structures/likely-min-energy-structures/zz_CSVff-[0,1,0]-L-0.13114_0-U-0.86904_0-ss-32.5374871816.cif
No. of atoms in the structure: 744
../structures/likely-min-energy-structures/zz_EONff-[0,0,1]-L-0.248379_0-U-0.751671_0-ss-9.93437482244.cif
No. of atoms in the structure: 336
../structures/likely-min-energy-structures/zz_EEIff-[0,0,1]-L-0.000335_0-U-0.999742_0-ss-63.1880627716.cif
../struct

No. of atoms in the structure: 240
../structures/likely-min-energy-structures/zz_FAUff-[1,1,1]-L-0.108313_0-U-0.891814_0-ss-28.2598506696.cif
No. of atoms in the structure: 4896
../structures/likely-min-energy-structures/zz_EEIff-[0,0,1]-L-0.378148_0-U-0.621631_0-ss-24.5046832252.cif
No. of atoms in the structure: 2784
../structures/likely-min-energy-structures/zz_ITGff-[1,0,1]-L-0.461433_0-U-0.461433_0-ss-12.1824832347.cif
No. of atoms in the structure: 768
../structures/likely-min-energy-structures/zz_GIUff-[1,0,0]-L-0.080945_0-U-0.917323_0-ss-18.7550396258.cif
No. of atoms in the structure: 2136
../structures/likely-min-energy-structures/zz_ETRff-[1,0,0]-L-0.255124_0-U-0.744898_0-ss-30.0336617534.cif
No. of atoms in the structure: 1008
../structures/likely-min-energy-structures/zz_ITRff-[0,1,0]-L-0.253626_0-U-0.631639_0-ss-8.7298953877.cif
No. of atoms in the structure: 672
../structures/likely-min-energy-structures/zz_EUOff-[0,1,0]-L-0.372403_0-U-0.749787_0-ss-53.0596694021.cif
No.

...

No. of atoms in the structure: 228
../structures/likely-min-energy-structures/zz_OBWff-[0,0,1]-L-0.249333_0-U-0.751674_0-ss-48.1529608105.cif
No. of atoms in the structure: 1440
../structures/likely-min-energy-structures/zz_OSOff-[1,0,0]-L-0.278456_0-U-0.720353_0-ss-24.9138017709.cif
No. of atoms in the structure: 336
../structures/likely-min-energy-structures/zz_PUNff-[0,0,1]-L-0.148533_0-U-0.851773_0-ss-12.1182746463.cif
No. of atoms in the structure: 288
../structures/likely-min-energy-structures/zz_SFOff-[1,0,0]-L-0.069398_0-U-0.930483_0-ss-44.2027497747.cif
No. of atoms in the structure: 792
../structures/likely-min-energy-structures/zz_SFGff-[0,1,0]-L-0.189883_0-U-0.810364_0-ss-30.2087400275.cif
No. of atoms in the structure: 2136
../structures/likely-min-energy-structures/zz_SFOff-[1,0,0]-L-0.069398_0-U-0.930483_0-ss-66.3031632941.cif
No. of atoms in the structure: 1176
../structures/likely-min-energy-structures/zz_SBSff-[0,0,1]-L-0.194888_0-U-0.896529_0-ss-19.1305388885.cif
No.

No. of atoms in the structure: 912
../structures/likely-min-energy-structures/zz_SBTff-[0,0,1]-L-0.068974_0-U-0.870132_0-ss-116.13843828.cif
No. of atoms in the structure: 4824
../structures/likely-min-energy-structures/zz_STOff-[1,0,-1]-L-0.448986_0-U-0.448986_0-ss-22.0751910557.cif
No. of atoms in the structure: 1440
../structures/likely-min-energy-structures/zz_PUNff-[0,0,1]-L-0.602442_1-U-0.748421_0-ss-21.607384437.cif
No. of atoms in the structure: 480
../structures/likely-min-energy-structures/zz_PUNff-[0,0,1]-L-0.602442_1-U-0.748421_0-ss-40.632558147.cif
No. of atoms in the structure: 912
../structures/likely-min-energy-structures/zz_PUNff-[0,0,1]-L-0.849869_0-U-0.649421_0-ss-17.9665369441.cif
No. of atoms in the structure: 456
../structures/likely-min-energy-structures/zz_STOff-[1,0,-1]-L-0.448986_0-U-0.448986_0-ss-43.455956239.cif
No. of atoms in the structure: 2784
../structures/likely-min-energy-structures/zz_PUNff-[0,0,1]-L-0.849869_0-U-0.649421_0-ss-36.9917106541.cif
No. o

No. of atoms in the structure: 768
../structures/likely-min-energy-structures/zz_MTWff-[2,0,-1]-L-0.631605_0-U-0.631605_0-ss-10.6514273171.cif
No. of atoms in the structure: 384
../structures/likely-min-energy-structures/zz_MTWff-[2,0,-1]-L-0.631605_0-U-0.631605_0-ss-20.8324798272.cif
No. of atoms in the structure: 720
../structures/likely-min-energy-structures/zz_MVYff-[1,1,0]-L-0.373809_0-U-0.373809_0-ss-15.001138503.cif
No. of atoms in the structure: 504
../structures/likely-min-energy-structures/zz_SEWff-[1,0,0]-L-0.050178_0-U-0.949875_0-ss-73.1194211633.cif
No. of atoms in the structure: 2424
../structures/likely-min-energy-structures/zz_TONff-[1,1,0]-L-0.247259_0-U-0.752792_0-ss-19.30072727.cif
No. of atoms in the structure: 576
../structures/likely-min-energy-structures/zz_STWff-[1,0,2]-L-0.672969_0-U-0.672969_0-ss-19.625764107.cif
No. of atoms in the structure: 1584
../structures/likely-min-energy-structures/zz_TONff-[1,1,0]-L-0.247259_0-U-0.752792_0-ss-30.154686437.cif
No. of 

No. of atoms in the structure: 888
../structures/likely-min-energy-structures/zz_SZRff-[1,1,0]-L-0.531574_0-U-0.468556_0-ss-13.2076313113.cif
No. of atoms in the structure: 480
../structures/likely-min-energy-structures/zz_SZRff-[1,1,0]-L-0.531574_0-U-0.468556_0-ss-24.4286488019.cif
No. of atoms in the structure: 912
../structures/likely-min-energy-structures/zz_SZRff-[1,1,0]-L-0.717458_0-U-0.282672_0-ss-20.4554211607.cif
No. of atoms in the structure: 888
../structures/likely-min-energy-structures/zz_OBWff-[0,0,1]-L-0.249916_0-U-0.7504_0-ss-17.3633448809.cif
No. of atoms in the structure: 504
../structures/likely-min-energy-structures/zz_SZRff-[1,1,0]-L-0.717458_0-U-0.282672_0-ss-9.23440367005.cif
No. of atoms in the structure: 456
../structures/likely-min-energy-structures/zz_TERff-[0,0,1]-L-0.327534_0-U-0.827523_0-ss-29.7275831517.cif
No. of atoms in the structure: 1512
../structures/likely-min-energy-structures/zz_TERff-[0,0,1]-L-0.327534_0-U-0.827523_0-ss-49.5457341017.cif
No. of 

No. of atoms in the structure: 576
../structures/likely-min-energy-structures/zz_UOZff-[1,0,0]-L-0.001962_0-U-0.001962_0-ss-19.416138447.cif
No. of atoms in the structure: 1056
../structures/likely-min-energy-structures/zz_UOZff-[1,0,0]-L-0.001962_0-U-0.001962_0-ss-29.7631712336.cif
No. of atoms in the structure: 1536
../structures/likely-min-energy-structures/zz_UOZff-[1,0,0]-L-0.001962_0-U-0.001962_0-ss-38.3102068636.cif
No. of atoms in the structure: 2016
../structures/likely-min-energy-structures/zz_USIff-[1,0,0]-L-0.075304_0-U-0.924812_0-ss-20.3385799779.cif
No. of atoms in the structure: 504
../structures/likely-min-energy-structures/zz_USIff-[1,0,0]-L-0.075304_0-U-0.924812_0-ss-40.6770786018.cif
No. of atoms in the structure: 984
../structures/likely-min-energy-structures/zz_VNIff-[1,1,1]-L-0.10208_0-U-0.897431_0-ss-8.33818413477.cif
No. of atoms in the structure: 816
../structures/likely-min-energy-structures/zz_USIff-[1,0,0]-L-0.075304_0-U-0.924812_0-ss-61.0155772256.cif
No. o

In [ ]:
import networkx as nx
g = nx.convert_matrix.from_numpy_array((mat>0.99)*1)
comps = nx.connected_components(g)

In [ ]:
[len(c) for c in sorted(comps, key=len, reverse=True)]

In [ ]:
nx.draw(g)
plt.show()

In [ ]:
plt.imshow((mat>0.99)*1)
plt.colorbar()
plt.show()